 - https://stackoverflow.com/questions/59557146/modifying-the-date-index-of-pandas-dataframe

In [1]:
import pandas as pd
import numpy as np
import time
from pandas.tseries.offsets import MonthEnd

In [2]:
def year_replace(st_d, end_d, x):

    tmp = time.perf_counter()

    def prior_year(d):
        # 100 is number of the years back, more than enough.
        for i_t in range(100):

            #The month should have been fully seen in one of the data years.
            t_start = pd.to_datetime(str(d.month) + '/' + str(end_d.year - i_t), format="%m/%Y")
            t_end = t_start + MonthEnd(1)
            if t_start <= end_d and t_start >= st_d and t_end <= end_d and t_end >= st_d:
                break
        if i_t < 99:
            return t_start.year
        else:
            raise BadDataException("Not enough data for Gradient Boosted tree.")

    output = pd.Series(index = x.index, data = x.index.map(lambda tt: prior_year(tt)), name = 'year')

    print("time for single dataframe replacement = ", time.perf_counter() - tmp)    

    return output

In [3]:
i = pd.date_range('01-01-2019', '01-01-2020')
x = pd.DataFrame(index = i, data=np.full(len(i), 0))

In [4]:
st_d = pd.to_datetime('01/2016', format="%m/%Y")
end_d = pd.to_datetime('01/2018', format="%m/%Y")
year_replace(st_d, end_d, x)

time for single dataframe replacement =  0.18637839999999528


2019-01-01    2017
2019-01-02    2017
2019-01-03    2017
2019-01-04    2017
2019-01-05    2017
              ... 
2019-12-28    2017
2019-12-29    2017
2019-12-30    2017
2019-12-31    2017
2020-01-01    2017
Freq: D, Name: year, Length: 366, dtype: int64

In [43]:
def f(start, stop, x):
    assert start < stop
    tmp = time.perf_counter()
    def y(d):
        # Current Year is ok:
        if start <= d.replace(day=1, year=stop.year) <= stop:
            return stop.year
        # Previous Year
        if start <= d.replace(day=1, year=stop.year-1) <= stop:
            return stop.year-1
        raise TypeError("Ooops")
    df = pd.Series(index=x.index, data=x.index.map(lambda t: y(t)), name='year')
    print("Tick: ", time.perf_counter() - tmp) 
    return df

In [45]:
def f(start, stop, x):
    assert start < stop
    tmp = time.perf_counter()
    def y(d):
        # Current Year is ok:
        if start < d.replace(year=stop.year) < stop:
            return stop.year
        # Previous Year
        if start < d.replace(year=stop.year-1) < stop:
            return stop.year-1
        raise TypeError("Ooops")
    df = pd.Series(index=x.index, data=x.index.map(lambda t: y(t)), name='year')
    print("Tick: ", time.perf_counter() - tmp) 
    return df

In [49]:
all(f(st_d, end_d, x) == year_replace(st_d, end_d, x))

Tick:  0.007427600000028178
time for single dataframe replacement =  0.16242110000030152


True